In [2]:
import pandas as pd
import plotly.express as px

In [100]:
df_tren = pd.read_csv("../data/external/transporte_publico/trenes/stops.txt")
df_subte = pd.read_csv("../data/external/transporte_publico/subte/estaciones-de-subte.csv")
df_premetro = pd.read_csv("../data/external/transporte_publico/premetro/estaciones-premetro.csv")

In [101]:
df_tren["type"] = "tren"
# df_tren = df_tren.drop(["stop_id"], axis=1)
df_tren = df_tren.rename(columns={"stop_lat": "lat", "stop_lon": "lon"})

In [102]:
df_tren.head()

,stop_id,stop_name,lat,lon,type
0,154,González Catán,-34.771464,-58.646546,tren
1,178,Independencia,-34.760798,-58.617341,tren
2,180,Ing. Budge,-34.718663,-58.462152,tren
3,184,Isidro Casanova,-34.704464,-58.587990,tren
4,203,Justo Villegas,-34.708965,-58.551939,tren


In [103]:
df_subte["type"] = "subte"
df_subte = df_subte.drop(["linea"], axis=1)
df_subte = df_subte.rename(columns={"long": "lon","estacion":"stop_name", "id": "stop_id"})
df_subte["stop_id"] = df_subte["stop_id"].astype(int)
df_subte = df_subte[df_tren.columns]

In [104]:
df_subte.head()

,stop_id,stop_name,lat,lon,type
0,1,CASEROS,-34.635750,-58.398928,subte
1,2,INCLAN - MEZQUITA AL AHMAD,-34.629376,-58.400970,subte
2,3,HUMBERTO 1°,-34.623092,-58.402323,subte
3,4,VENEZUELA,-34.615242,-58.404732,subte
4,5,ONCE - 30 DE DICIEMBRE,-34.608935,-58.406036,subte


In [106]:
df_premetro["type"] = "premetro"
# df_premetro = df_premetro.drop(["id"], axis=1)
df_premetro = df_premetro.rename(columns={"long": "lon","nombre":"stop_name", "id": "stop_id"})
df_premetro = df_premetro[df_tren.columns]

In [107]:
df_premetro.head()

,stop_id,stop_name,lat,lon,type
0,1,INTENDENTE SAGUIER,-34.643757,-58.461491,premetro
1,2,BALBASTRO (Cementerio de Flores),-34.648601,-58.456415,premetro
2,3,ANA MARÍA JANER,-34.659499,-58.450047,premetro
3,4,FERNANDEZ DE LA CRUZ,-34.662225,-58.446607,premetro
4,5,PTE. ILLIA (Lacarra),-34.665368,-58.448513,premetro


In [108]:
df = pd.concat([df_tren, df_subte, df_premetro])

In [109]:
df.head()

,stop_id,stop_name,lat,lon,type
0,154,González Catán,-34.771464,-58.646546,tren
1,178,Independencia,-34.760798,-58.617341,tren
2,180,Ing. Budge,-34.718663,-58.462152,tren
3,184,Isidro Casanova,-34.704464,-58.587990,tren
4,203,Justo Villegas,-34.708965,-58.551939,tren


In [112]:
df["lat"] = df["lat"].astype(float)
df["lon"] = df["lon"].astype(float)

In [116]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_data=["stop_name", "stop_id"],
                        color="type",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [157]:
centro_geografico_caba = [-34.61448064637565, -58.44638197269814]
centro_geografico_plata = [-34.92137784728068, -57.95455274162061]

In [119]:
from geopy import distance

In [158]:
df["dist_centro_caba"] = df.apply(lambda x: distance.distance(centro_geografico_caba, [x.lat, x.lon]).km, axis=1)
df["dist_centro_plata"] = df.apply(lambda x: distance.distance(centro_geografico_plata, [x.lat, x.lon]).km, axis=1)


In [165]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df[(df.dist_centro_caba < 11) | (df.dist_centro_plata < 17)],
                        lat="lat",
                        lon="lon",
                        hover_data=["stop_name", "stop_id"],
                        color="type",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [142]:
df_routes = pd.read_csv("../data/external/transporte_publico/trenes/routes.txt")
df_routes = df_routes[["route_id", "route_short_name", "route_long_name"]]
df_routes = df_routes.drop_duplicates(keep="last")
df_routes.head(2)

,route_id,route_short_name,route_long_name
0,1,Sarmiento,Sarmiento / Once - Moreno
1,5,Mitre,Mitre / Retiro - Tigre


In [143]:
df_trips = pd.read_csv("../data/external/transporte_publico/trenes/trips.txt")
df_trips = df_trips[['route_id',  'trip_id']]
df_trips = df_trips.drop_duplicates(keep="last")
df_trips.head(2)

,route_id,trip_id
0,1,32001211
1,1,32011111


In [144]:
df_stop_times = pd.read_csv("../data/external/transporte_publico/trenes/stop_times.txt")
df_stop_times = df_stop_times[['trip_id', 'stop_id']]
df_stop_times = df_stop_times.drop_duplicates(keep="last")
df_stop_times.head(2)

,trip_id,stop_id
0,32011111,70
1,32011111,186


In [145]:
df = df_routes.merge(df_trips, on="route_id").merge(df_stop_times, on="trip_id").merge(df_tren, on="stop_id")

In [146]:
df.head(2)

,route_id,route_short_name,route_long_name,trip_id,stop_id,stop_name,lat,lon,type
0,1,Sarmiento,Sarmiento / Once - Moreno,32001211,70,Castelar,-34.652156,-58.642526,tren
1,1,Sarmiento,Sarmiento / Once - Moreno,32011111,70,Castelar,-34.652156,-58.642526,tren


In [147]:
df = df[["route_id", "route_short_name", "route_long_name", "stop_id", "stop_name", "lat", "lon", "type"]]

In [149]:
df = df.drop_duplicates(keep="last")

In [150]:
df

,route_id,route_short_name,route_long_name,stop_id,stop_name,lat,lon,type
625,1,Sarmiento,Sarmiento / Once - Moreno,70,Castelar,-34.652156,-58.642526,tren
1241,1,Sarmiento,Sarmiento / Once - Moreno,279,Morón,-34.648223,-58.619813,tren
1858,1,Sarmiento,Sarmiento / Once - Moreno,169,Haedo,-34.644533,-58.591853,tren
2454,1,Sarmiento,Sarmiento / Once - Moreno,327,Ramos Mejía,-34.640454,-58.564376,tren
3050,1,Sarmiento,Sarmiento / Once - Moreno,84,Ciudadela,-34.639923,-58.541123,tren
...,...,...,...,...,...,...,...,...
69149,65,San Martin,San Martín / Retiro (LSM) - Cabred,369,Santos Lugares,-34.603674,-58.540752,tren
69577,65,San Martin,San Martín / Retiro (LSM) - Cabred,112,Devoto,-34.602652,-58.512431,tren
69950,65,San Martin,San Martín / Retiro (LSM) - Cabred,297,Palermo,-34.579661,-58.427415,tren
70323,65,San Martin,San Martín / Retiro (LSM) - Cabred,463,Retiro LSM,-34.588130,-58.375689,tren


In [154]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_data=["route_short_name", "route_long_name", "stop_name", "stop_id"],
                        color="type",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [25]:
df = pd.read_csv("../data/external/other/hospitales.csv")

In [26]:
df.head()

,WKT,ID,NOMBRE,NOM_MAP,OBJETO,CALLE,ALTURA,DOM_NORMA,DOM_GEO,TELEFONO,GUARDIA,FAX,WEB,COD_POSTAL,TIPO,TIPO_ESPEC,MOD_AT_1,MOD_AT_2,DEPEND_ADM,DIRECTOR
0,POINT (-58.3775508488443 -34.6288473603881),1.0,HOSPITAL GENERAL DE NIÑOS PEDRO DE ELIZALDE,HOSP. DE ELIZALDE,HOSPITAL,MANUEL A. MONTES DE OCA,40,"MONTES DE OCA, MANUEL AV. 40","40 MONTES DE OCA, MANUEL AV.",4307-5842 / 5844,4307-5442 / 4300-1700,4302-7400,www.elizalde.gov.ar,C1270AAN,Hospital de niños,PEDIATRIA,AT. AMB./INTERNACION,DIAG./TRATAM.,Ministerio de Salud GCBA,Dr. Javier Indart
1,POINT (-58.4120700769456 -34.5941919726393),2.0,HOSPITAL GENERAL DE NIÑOS RICARDO GUTIERREZ,HOSP. GUTIERREZ,HOSPITAL,GALLO,1330,GALLO 1330,1330 GALLO,4962-9247 / 9248 / 9280,4962-9232,4962-3762,www.guti.gov.ar,C1425EFD,Hospital de niños,PEDIATRIA,AT. AMB./INTERNACION,DIAG./TRATAM.,Ministerio de Salud GCBA,Dra. María Cristina Galoppo
2,POINT (-58.4027276547828 -34.5845283357505),4.0,HOSPITAL DE ODONTOLOGIA DR. RAMON CARRILLO (EX...,HOSP. ODONTOLOGICO CARRILLO,HOSPITAL,SANCHEZ DE BUSTAMANTE,2529,SANCHEZ DE BUSTAMANTE 2529,2529 SANCHEZ DE BUSTAMANTE,4805-5521 / 7533,4805-6407,4805-7533,NaN,C1425DUY,Hospital especializado,ODONTOLOGIA,AT. AMBULATORIA,DIAG./TRATAM.,Ministerio de Salud GCBA,Dra. Susana Rita Lisanti
3,POINT (-58.3851559118889 -34.6394041316484),6.0,HOSPITAL DE SALUD MENTAL BRAULIO MOYANO,HOSP. MOYANO,HOSPITAL,BRANDSEN,2570,BRANDSEN 2570,2570 BRANDSEN,4301-3655 / 3659,4301-4522,4303-3655/59,www.moyano.org.ar,C1287ABJ,Hospital especializado,SALUD MENTAL,AT. AMB./INTERNACION,DIAG./TRATAM.,Ministerio de Salud GCBA,Dra. Norma Derito
4,POINT (-58.3913114406535 -34.6341535760002),7.0,HOSPITAL DE GASTROENTEROLOGIA B. UDAONDO,HOSP. UDAONDO,HOSPITAL,CASEROS,2061,CASEROS AV. 2061,2061 CASEROS AV.,4306-4641 al 4649,4306-4641 al 49 (int. 125),4306-3013,NaN,C1264AAA,Hospital especializado,ENF. APAR. DIGESTIVO,AT. AMB./INTERNACION,DIAG./TRATAM.,Ministerio de Salud GCBA,Dr. Eduardo Gustavo Sosa


In [28]:
import json

In [29]:
with open("../data/external/other/hospitales.json") as f:
    hospitales = json.load(f)

In [34]:
hospitales["features"]

[{'type': 'Feature',
  'properties': {'ID': 1.0,
   'NOMBRE': 'HOSPITAL GENERAL DE NIÑOS PEDRO DE ELIZALDE',
   'NOM_MAP': 'HOSP. DE ELIZALDE',
   'OBJETO': 'HOSPITAL',
   'CALLE': 'MANUEL A. MONTES DE OCA',
   'ALTURA': '40',
   'DOM_NORMA': 'MONTES DE OCA, MANUEL AV. 40',
   'DOM_GEO': '40 MONTES DE OCA, MANUEL AV.',
   'TELEFONO': '4307-5842 / 5844',
   'GUARDIA': '4307-5442 / 4300-1700',
   'FAX': '4302-7400',
   'WEB': 'www.elizalde.gov.ar',
   'COD_POSTAL': 'C1270AAN',
   'TIPO': 'Hospital de niños',
   'TIPO_ESPEC': 'PEDIATRIA',
   'MOD_AT_1': 'AT. AMB./INTERNACION',
   'MOD_AT_2': 'DIAG./TRATAM.',
   'DEPEND_ADM': 'Ministerio de Salud GCBA',
   'DIRECTOR': 'Dr. Javier Indart'},
  'geometry': {'type': 'Point',
   'coordinates': [-58.37755084884431, -34.628847360388136]}},
 {'type': 'Feature',
  'properties': {'ID': 2.0,
   'NOMBRE': 'HOSPITAL GENERAL DE NIÑOS RICARDO GUTIERREZ',
   'NOM_MAP': 'HOSP. GUTIERREZ',
   'OBJETO': 'HOSPITAL',
   'CALLE': 'GALLO',
   'ALTURA': '1330',
 

In [38]:
df = pd.DataFrame(hospitales["features"])

In [56]:
df["lon"] = df["geometry"].apply(lambda x: x["coordinates"][0])
df["lat"] = df["geometry"].apply(lambda x: x["coordinates"][1])
df["name"] = df["properties"].apply(lambda x: x["NOMBRE"])

In [57]:
df.head(2)

,type,properties,geometry,lat,lon,name
0,Feature,"{'ID': 1.0, 'NOMBRE': 'HOSPITAL GENERAL DE NIÑ...","{'type': 'Point', 'coordinates': [-58.37755084...",-34.628847,-58.377551,HOSPITAL GENERAL DE NIÑOS PEDRO DE ELIZALDE
1,Feature,"{'ID': 2.0, 'NOMBRE': 'HOSPITAL GENERAL DE NIÑ...","{'type': 'Point', 'coordinates': [-58.41207007...",-34.594192,-58.412070,HOSPITAL GENERAL DE NIÑOS RICARDO GUTIERREZ


In [58]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_name="name",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [89]:
df = pd.read_csv("../data/external/other/hospitales_LaPlata.csv", delimiter=";", encoding='latin1')

In [ ]:
df = df.drop(["DIRECCION", "TELEFONO", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7"], axis=1)
df = df.rename(columns={"LATITUD": "lat", "LONGITUD": "lon", "CLINICAS Y HOSPITALES": "name"})
df["lat"] = df["lat"].str.replace(",", ".").replace(" ", "").astype(float)
df["lon"] = df["lon"].str.replace(",", ".").replace(" ", "").astype(float)
df["province"] = "Bs.As. G.B.A. Zona Sur"

In [90]:
df.head()

,name,lat,lon
0,BREAST CLINIC S.A.,-34.907611,-57.959000
1,CEMPLAN,-34.918250,-57.942444
2,"Centro Educación y Tratamiento para Diabetes ""...",-34.927278,-57.933917
3,Centro Gastroenterológico Platense,-34.913417,-57.942333
4,Centro Médico Cantilo,-34.880056,-58.053750


In [94]:
df.head(2)


,name,lat,lon
0,BREAST CLINIC S.A.,-34.907611,-57.959000
1,CEMPLAN,-34.918250,-57.942444


In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    43 non-null     object 
 1   lat     41 non-null     float64
 2   lon     41 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.1+ KB


In [93]:
color_scale = [(0, 'orange'), (1,'red')]

fig = px.scatter_mapbox(df,
                        lat="lat",
                        lon="lon",
                        hover_name="name",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

,stop_id,stop_name,lat,lon,type
0,154,González Catán,-34.771464,-58.646546,tren
1,178,Independencia,-34.760798,-58.617341,tren
2,180,Ing. Budge,-34.718663,-58.462152,tren
3,184,Isidro Casanova,-34.704464,-58.587990,tren
4,203,Justo Villegas,-34.708965,-58.551939,tren
...,...,...,...,...,...
13,14,ANA DÍAZ,-34.680883,-58.466923,premetro
14,15,MARIANO ACOSTA (Bo. R. Carrillo),-34.654433,-58.456480,premetro
15,16,SOMELLERA,-34.656441,-58.453905,premetro
16,17,NTRA. SRA. DE FÁTIMA,-34.660458,-58.448861,premetro
